In [ ]:
from google.colab import drive
deive.mount('/content/drive/')

In [ ]:
import os
print("Annots:",len(os.listdir("/content/drive/MyDrive/debris_VOCdevkit/VOCdevkit/VOC2007/Annotations")))
print("Images:",len(os.listdir("/content/drive/MyDrive/debris_VOCdevkit/VOCdevkit/VOC2007/JPEGImages")))

Annots: 11666
Images: 11674


In [ ]:
%cd '/content/drive/MyDrive/debris_VOCdevkit/VOCdevkit/VOC2007'

/content/drive/MyDrive/debris_VOCdevkit/VOCdevkit/VOC2007


In [ ]:
ls

Annotations/  ImageSets/  JPEGImages/


In [ ]:
classes = []
input_dir = "Annotations/"
output_dir = "labels/"
image_dir = "JPEGImages/"
#os.mkdir(output_dir)

## xml to yolo labels 

In [ ]:
import xml.etree.ElementTree as ET
import glob
import os
import json

In [ ]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

In [ ]:
# identify all the xml files in the annotations folder (input directory)
files = glob.glob(os.path.join(input_dir, '*.xml'))
# loop through each 
for i in range(len(files)):
    basename = os.path.basename(files[i])
    filename = os.path.splitext(basename)[0]
    # check if the label contains the corresponding image file
    #if not os.path.exists(os.path.join(image_dir, f"{filename}.jpg")):
        #print(f"{filename} image does not exist!")
        #continue

    result = []

    # parse the content of the xml file
    tree = ET.parse(files[i])
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    for obj in root.findall('object'):
        label = obj.find("name").text
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")

    if result:
        # generate a YOLO format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
            f.write("\n".join(result))

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))

In [ ]:
len(os.listdir("labels"))

11666

## Train & Test

In [ ]:
import random
import shutil

In [ ]:
ls

Annotations/  classes.txt  ImageSets/  JPEGImages/  labels/


In [ ]:
def copyfiles(fil, root_dir):
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]

    # copy image
    src_img = os.path.join(image_dir,fil)
    dest_img = os.path.join(root_dir, image_dir, fil)
    
    # copy annotations
    src = os.path.join(label_dir, f"{filename}.txt")
    dest = os.path.join(root_dir, label_dir, f"{filename}.txt")
    
    if os.path.exists(src):
        shutil.copyfile(src_img, dest_img)
        shutil.copyfile(src, dest)


label_dir = "labels/"
image_dir = "JPEGImages/"
lower_limit = 0
files = os.listdir(image_dir)

random.shuffle(files)

folders = {"train": 0.8, "val": 0.1, "test": 0.1}
check_sum = sum([folders[x] for x in folders])

assert check_sum == 1.0, "Split proportion is not equal to 1.0"

for folder in folders:
    os.mkdir(folder)
    temp_label_dir = os.path.join(folder, label_dir)
    os.mkdir(temp_label_dir)
    temp_image_dir = os.path.join(folder, image_dir)
    os.mkdir(temp_image_dir)

    limit = round(len(files) * folders[folder])
    for fil in files[lower_limit:lower_limit + limit]:
        copyfiles(fil, folder)
    lower_limit = lower_limit + limit

In [ ]:
len(os.listdir('train/JPEGImages')),len(os.listdir('train/labels'))

(9310, 9310)

In [ ]:
len(os.listdir('test/JPEGImages')),len(os.listdir('test/labels'))

(1162, 1162)

In [ ]:
len(os.listdir('val/JPEGImages')),len(os.listdir('val/labels'))

(1163, 1163)

In [ ]:
L=["Natural_debris", "Metal", "Plastic", "Fishing_net_Rope", "Cloth", "Rubber", "Glass"]

In [ ]:
len(L)

7